In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
sys.path.append(os.path.join(os.path.pardir, 'gotmtool'))
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from gotmtool import *
from lesview import *
from sbl_bbl import *

In [ ]:
g = 9.81
H = 30
u10 = 8
N2 = 1.962e-4
bstar = N2 * H
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
lat = 45.
alphaT = 2.0e-4
tau = rhoa/rhoo*cd*u10*u10
ustar = np.sqrt(tau)
amplitude = 1.0
wavelength = 60
wavenumber = 2.*np.pi/wavelength
frequency = np.sqrt(g*wavenumber*np.tanh(wavenumber*H))
us0 = amplitude**2*wavenumber
la = np.sqrt(ustar/us0) 
print('La = {:6.3f}'.format(la))
Ti = inertial_period(lat)
print('Ti = {:6.3f}'.format(Ti))

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_v2'
turbmethods = ['SMCLT-H15', 'SMCLT-KC04', 'KPPLT-LF17']

runs = {'r1': '', 'r2': '_rf'}
nlev = 144
rlx = 'Rlx0'
ds_gotm = {}
ds_ocgn = {}
for rkey in runs.keys():
    datapath = os.path.join(os.path.pardir, 'oceananigans', '{:s}{:s}'.format(casename, runs[rkey]))
    filepath = os.path.join(datapath, 'averages.jld2')
    data_pfl = OceananigansDataProfile(filepath=filepath)
    ds_ocgn[rkey] = data_pfl.dataset
    tmp = {}
    gotm_dir = os.path.join(os.path.pardir, 'gotm', 'run', '{:s}{:s}'.format(casename, runs[rkey]))
    for turbmethod in turbmethods:
        gotm_sim = Simulation(path=os.path.join(gotm_dir, '{:s}_L{:g}_{:s}'.format(turbmethod, nlev, rlx)))
        tmp[turbmethod] = gotm_sim.load_data()
    ds_gotm[rkey] = tmp

In [ ]:
figpath  = 'overview_{:s}'.format(casename)
os.makedirs(figpath, exist_ok=True)

In [ ]:
def get_das(ds):
    das = dict(
            NN = ds.data_vars['temp'].differentiate(coord='z')*alphaT*g/N2,
            SS = ((ds.data_vars['u']+ds.data_vars['us']).differentiate(coord='z')**2+(ds.data_vars['v']+ds.data_vars['vs']).differentiate(coord='z')**2)/N2,
            # Ri = ds.data_vars['temp'].differentiate(coord='z')*alphaT*g/((ds.data_vars['u']+ds.data_vars['us']).differentiate(coord='z')**2+(ds.data_vars['v']+ds.data_vars['vs']).differentiate(coord='z')**2),
            Ri = (ds.data_vars['temp'].differentiate(coord='z')*alphaT*g-0.25*((ds.data_vars['u']+ds.data_vars['us']).differentiate(coord='z')**2+(ds.data_vars['v']+ds.data_vars['vs']).differentiate(coord='z')**2))/N2,       
            wb = get_flux(ds.data_vars['temp'].squeeze(),
                          ds.data_vars['nuh'].squeeze(),
                          ds.data_vars['gamh'].squeeze())*alphaT*g/ustar/bstar*1e3,
            wu = (get_flux(ds.data_vars['u'].squeeze(),
                          ds.data_vars['num'].squeeze(),
                          ds.data_vars['gamu'].squeeze())
                  -ds.data_vars['nucl'].squeeze()*ds.data_vars['dusdz'].squeeze())/ustar**2,
            wv = (get_flux(ds.data_vars['v'].squeeze(),
                          ds.data_vars['num'].squeeze(),
                          ds.data_vars['gamv'].squeeze())
                  -ds.data_vars['nucl'].squeeze()*ds.data_vars['dvsdz'].squeeze())/ustar**2,
        )
    return das

In [ ]:
edges1_les = {}
edges2_les = {}
for k, rkey in enumerate(runs.keys()):
    edges1_les[rkey], edges2_les[rkey] = get_edges(nondim_da(ds_ocgn[rkey].data_vars['b'].differentiate(coord='z')/N2, H=H, Tf=Ti))

In [ ]:
edges1_gotm = {}
edges2_gotm = {}
for k, rkey in enumerate(runs.keys()):
    tmp1 = {}
    tmp2 = {}
    for turbmethod in turbmethods:
        tmp1[turbmethod], tmp2[turbmethod] = get_edges(nondim_da(ds_gotm[rkey][turbmethod].data_vars['temp'][:,:,0,0].differentiate(coord='z')*alphaT*g/N2, H=H, Tf=Ti))
    edges1_gotm[rkey] = tmp1
    edges2_gotm[rkey] = tmp2

In [ ]:
labels = dict(
    NN = '$N^2/N_0^2$',
    SS = '$S^2/N_0^2$',
    # Ri = '$N^2/S^2$',
    Ri = '$(4N^S-S^2)/4N_0^2$',
    wb = '$10^3\overline{w^\prime b^\prime}/u_*b_*$',
    wu = '$\overline{w^\prime u^\prime}/u_*^2$',
    wv = '$\overline{w^\prime v^\prime}/u_*^2$',
)
levels = dict(
    NN = np.linspace(0, 4, 41),
    SS = np.linspace(0, 8, 41),
    # Ri = np.linspace(0, 1, 5),
    Ri = np.linspace(-3, 3, 13),
    wb = np.linspace(-4, 4, 41),
    wu = np.linspace(-2, 2, 41),
    wv = np.linspace(-1.2, 1.2, 41),
)
cmaps = dict(
    NN = 'pink_r',
    SS = 'pink_r',
    # Ri = 'pink_r',
    Ri = 'RdBu_r',
    wb = 'RdBu_r',
    wu = 'RdBu_r',
    wv = 'RdBu_r',
)
abc = dict(
    NN = 'ag',
    SS = 'bh',
    Ri = 'ci',
    wb = 'dj',
    wu = 'ek',
    wv = 'fl',
)
tags = ['Aligned', 'Opposite']
nv = int(len(labels))
rlcolor = {'RdBu_r': 'k', 'viridis': 'w', 'bone': 'w', 'bone_r': 'k', 'pink_r': 'k'}
tmcolor = {'SMCLT-KC04': 'tab:red', 'KPPLT-LF17': 'tab:purple'}
line_kwargs = dict(color='k', linewidth=0.75)
turbmethod = 'SMCLT-H15'
cf = {}
fig, axarr = plt.subplots(nv, 2, sharex='col')
fig.set_size_inches([10, 0.4+1.5*nv])
for k, rkey in enumerate(runs.keys()):
    ds = ds_gotm[rkey][turbmethod]
    das = nondim_das(get_das(ds), H=H, Tf=Ti)
    z0, z1, tstart1, tend1, tstart2, tend2 = get_merge(das['NN'][:,:,0,0], Tf=Ti)
    for i, var in enumerate(das.keys()):
        ax = axarr[i, k]
        cmap = cmaps[var]
        cf[var] = das[var].plot(ax=ax, levels=levels[var], cmap=cmap, extend='both', add_colorbar=False)
        ax.axvline(x=tstart1, linestyle=':', color=rlcolor[cmap])
        ax.axvline(x=tend1,   linestyle=':', color=rlcolor[cmap])
        ax.axvline(x=tstart2, linestyle=':', color=rlcolor[cmap])
        ax.axvline(x=tend2,   linestyle=':', color=rlcolor[cmap])
        if i == 0:
            ax.text((tstart1+tend1)/2, -0.05, 'T1', va='top', ha='center')
            ax.text((tstart2+tend2)/2, -0.05, 'T2', va='top', ha='center')
            for eg in [edges1_les[rkey], edges2_les[rkey]]:
                eg.plot(ax=ax, color='gray', linewidth=1.5)
            for tm in ['SMCLT-KC04', 'KPPLT-LF17']:
                for eg in [edges1_gotm[rkey][tm], edges2_gotm[rkey][tm]]:
                    eg.plot(ax=ax, color=tmcolor[tm], linewidth=0.75)
        z0.plot(ax=ax, **line_kwargs)
        z1.plot(ax=ax, **line_kwargs)
        ax.text(0.98, 0.92, '({:s})'.format(abc[var][k]), transform=ax.transAxes, va='top', ha='right')
        ax.set_title('')
        ax.set_xlim([0,16])
        if i == nv-1:
            ax.set_xlabel('$t/T_f$')
        else:
            ax.set_xlabel('')
        if k == 0:
            ax.set_ylabel('$z/H$')
        else:
            ax.set_ylabel('')
    axarr[0,k].set_title(tags[k])
    
l1, = plt.plot(np.nan,np.nan,'gray', linewidth=1.5)
l2, = plt.plot(np.nan,np.nan,tmcolor['SMCLT-KC04'], linewidth=0.75)
l3, = plt.plot(np.nan,np.nan,tmcolor['KPPLT-LF17'], linewidth=0.75)
axarr[0,0].legend([l1,l2,l3],['LES','KC04','LF17'], loc='lower right', fontsize=9)

    
plt.subplots_adjust(top=0.97, bottom=0.06, left=0.08, right=0.87, hspace=0.15, wspace=0.15)
for i, var in enumerate(das.keys()):
    ax = axarr[i,1]
    pos = ax.get_position()
    cax = plt.axes([0.9, pos.y0, 0.015, pos.height])
    cb = plt.colorbar(cf[var], cax=cax)
    cb.set_label(labels[var])
figname = os.path.join(figpath, 'mean_gotm-v2-cmp-{:s}'.format(turbmethod))
fig.savefig(figname, dpi = 300, facecolor='w')